# Cassandra + Spark. Workshop.

Для работы в этом ноутбуке запустите jupyter, выполнив команду:  
```bash
sh /home/mf2019q2/mf_surname/pyspark-jupyter.sh $PORT1 $PORT2
```

## На этом семинаре:
+ знакомство с ```cqlsh```
+ создание таблиц
+ чтение и запись данных
+ работа с композитными ключами
+ использовать Spark Cassandra Connector

```cqlsh``` - основная утилита, которая позволяет подключаться к Cassandra в интерактивном режиме. Для подключения к кластеру просто выполните команду ```cqlsh virtual-node01``` в командной строке. Обычно при подключении к кластеру также требуется указать логин и пароль. Но на нашем тестовом стенде это не нужно :)

Также, используя флаг -e, можно выполнять запросы, не запуская интерактивного шелла, например:

In [2]:
%%bash
cqlsh virtual-node01 -e 'DESCRIBE KEYSPACE system'


CREATE KEYSPACE system WITH replication = {'class': 'LocalStrategy'}  AND durable_writes = true;

CREATE TABLE system.available_ranges (
    keyspace_name text PRIMARY KEY,
    ranges set<blob>
) WITH bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = 'available keyspace/ranges during bootstrap/replace that are ready to be served'
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND dclocal_read_repair_chance = 0.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 0
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 3600000
    AND min_index_interval = 128
    AND read_repair_chance = 0.0
    AND speculative_retry = '99PERCENTILE';

CREATE TABLE system.batches

Keyspace в Cassandra - это аналог database в классических БД. Любая таблица в Cassandra создается в одном из keyspace. В Cassandra есть несколько системных keyspace, например system, system_schema и system_auth. 

Фактор репликации (количество реплик) устанавливается на уровне keyspace. Создать keyspace можно с помощью следующей команды:

In [4]:
%%bash
cqlsh virtual-node01 -e \
"CREATE KEYSPACE IF NOT EXISTS mf WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3}"

В таблице system_schema.table содержится список всех таблиц в каждом keyspace. Выведем список всех таблиц в keyspace ```mf```

In [5]:
%%bash
cqlsh virtual-node01 -e \
"select keyspace_name, table_name from system_schema.tables WHERE keyspace_name = 'mf'"


 keyspace_name | table_name
---------------+------------
            mf |   students

(1 rows)


## Задача 1
Получите описание таблицы ```mf.students```:
+ Какие поля в ней присутствуют?
+ Какие типы у каждого поля?
+ Какое поле является ключевым?

Создайте новую запись в этой таблице с информацией о себе:
+ org - организация, в которой вы работаете ('megafon')
+ age - ваш возраст
+ first_name - имя
+ last_name - фамилия
+ student_id - уникальный идентификатор студента
+ job_role - ваша должность.

В качестве примера я заполнил данные о себе.

Вам не обязательно придумывать UUID. В Cassandra есть функция ```uuid()```, которая генеририует случайный UUID4

## Задача 2
Скооперируйтесь с коллегами и убедитесь, что все создали свои записи в ```mf.students```
Выведите содержмое таблицы на экран. 

+ все ли работаете как надо? Если нет, то почему? Исправьте ошибку.
+ продумайте схему данных, оптимизированную для выполнения следующего запроса: **какой максимальный возраст у сотрудников ```megafon``` с фамилией ```Ivanov```?**
+ напишите запрос, который будет отвечать на этот вопрос

## Задача 3
+ добавьте в таблицу новую колонку favourite_city text
+ заполните ее городом в США с большой буквы (например, Boston), который бы вы хотели посетить в ближайшем будущем

Добавить колонку сможет самый проворный :)

## Задача 4
Прочитайте знакомый вам датасет ```hdfs:///data/spark_excercise/nbagames.json``` с помощью Spark
Взорвите колонку teams и запишите данные в таблицу mf.games

Поле updated_at должно содержать текущий timestamp

Обратите внимание на следующие моменты:
+ возникают ли какие-то проблемы из-за параллельной записи разными людьми в одну таблицу?
+ создает ли это дубликаты данных?

## Задача 5
Прочитайте таблицу mf.games с помощью Spark.
Используя данные из обеих таблиц, дайте ответ на вопрос:
**Какая команда сыграла большего всего домашних игр в самом любимом городе студентов?**

+ за какое время время отрабатывает запрос?
+ что нужно изменить в схемах данных, чтобы скорость работы запроса выросла?
+ скопируйте таблицы (в конце имени новых таблиц поставьте ```_вашу_фамилию```) в таблицы с оптимальной структурой и повторите запрос